In [1]:
import pandas as pd
import datetime
import os
import sys
sys.path.append('../')
from data_processing import process_climate_data, save_data, load_nuts_boundaries, create_region_year_df, disaggregate_climate_from_nuts2_to_nuts3

Climate and energy indicators for Europe from 1979 to present derived from reanalysis  
link to data: https://cds.climate.copernicus.eu/cdsapp#!/dataset/sis-energy-derived-reanalysis?tab=overview


Meterology:
- Wind speed at 10m
- Surface downwelling shortwave radiation
- Pressure at sea level
- 2m air temperature
- Total precipitation

Spatial aggregation: Sub-country level

Temporal aggregation: Annual   
Resource updated: 2024-11-15  
Resource acceseed:2024-11-27

In [3]:
GHI = pd.read_csv(r"/gb_pv_capacity_model/data/20241127/raw/climate/H_ERA5_ECMW_T639_GHI_0000m_Euro_NUT2_S197901010000_E202410312300_INS_TIM_12m_NA-_noc_org_NA_NA---_NA---_NA---.csv", skiprows=52)
TP = pd.read_csv(r"/gb_pv_capacity_model/data/20241127/raw/climate/H_ERA5_ECMW_T639_TP-_0000m_Euro_NUT2_S197901010000_E202410312300_ACC_TIM_12m_NA-_noc_org_NA_NA---_NA---_NA---.csv", skiprows=52)
MSL = pd.read_csv(r"/gb_pv_capacity_model/data/20241127/raw/climate/H_ERA5_ECMW_T639_MSL_0000m_Euro_NUT2_S197901010000_E202410312300_INS_TIM_12m_NA-_noc_org_NA_NA---_NA---_NA---.csv", skiprows=52)
TA = pd.read_csv(r"/gb_pv_capacity_model/data/20241127/raw/climate/H_ERA5_ECMW_T639_TA-_0002m_Euro_NUT2_S197901010000_E202410312300_INS_TIM_12m_NA-_noc_org_NA_NA---_NA---_NA---.csv", skiprows=52)
WS = pd.read_csv(r"/gb_pv_capacity_model/data/20241127/raw/climate/H_ERA5_ECMW_T639_WS-_0010m_Euro_NUT2_S197901010000_E202410312300_INS_TIM_12m_NA-_noc_org_NA_NA---_NA---_NA---.csv", skiprows=52)


In [4]:
GHI = process_climate_data(r"/gb_pv_capacity_model/data/20241127/raw/climate/H_ERA5_ECMW_T639_GHI_0000m_Euro_NUT2_S197901010000_E202410312300_INS_TIM_12m_NA-_noc_org_NA_NA---_NA---_NA---.csv", 'ghi')

TP = process_climate_data(r"/gb_pv_capacity_model/data/20241127/raw/climate/H_ERA5_ECMW_T639_TP-_0000m_Euro_NUT2_S197901010000_E202410312300_ACC_TIM_12m_NA-_noc_org_NA_NA---_NA---_NA---.csv", 'total_precipitation')

MSL = process_climate_data(r"/gb_pv_capacity_model/data/20241127/raw/climate/H_ERA5_ECMW_T639_MSL_0000m_Euro_NUT2_S197901010000_E202410312300_INS_TIM_12m_NA-_noc_org_NA_NA---_NA---_NA---.csv", 'mean_sea_level_pressure')

TA = process_climate_data(r"/gb_pv_capacity_model/data/20241127/raw/climate/H_ERA5_ECMW_T639_TA-_0002m_Euro_NUT2_S197901010000_E202410312300_INS_TIM_12m_NA-_noc_org_NA_NA---_NA---_NA---.csv", 'air_temperature')

WS = process_climate_data(r"/gb_pv_capacity_model/data/20241127/raw/climate/H_ERA5_ECMW_T639_WS-_0010m_Euro_NUT2_S197901010000_E202410312300_INS_TIM_12m_NA-_noc_org_NA_NA---_NA---_NA---.csv", 'wind_speed')



In [5]:

GHI
TP
MSL
TA
WS


,date,nuts_cd,wind_speed
14214,1979-12-31,UKC1,4.392267
14215,1980-12-31,UKC1,4.316011
14216,1981-12-31,UKC1,4.334328
14217,1982-12-31,UKC1,4.377804
14218,1983-12-31,UKC1,4.564312
...,...,...,...
16095,2020-12-31,UKN0,4.487618
16096,2021-12-31,UKN0,3.941892
16097,2022-12-31,UKN0,4.314521
16098,2023-12-31,UKN0,4.187613


In [6]:
# Create a dictionary of DataFrames
dataframes = {'GHI': GHI, 'TP': TP, 'MSL': MSL, 'TA': TA, 'WS': WS}
# save as csv
save_data(dataframes)

### Disaggregate from NUTS 2 to NUTS 3

In [6]:
nuts = load_nuts_boundaries("/gb_pv_capacity_model/data/nuts.shp.zip", download=True)
nuts

,NUTS_ID,LEVL_CODE,CNTR_CODE,NAME_LATN,NUTS_NAME,MOUNT_TYPE,URBN_TYPE,COAST_TYPE,geometry
0,AL,0,AL,Shqipëria,Shqipëria,0.0,0,0,"MULTIPOLYGON (((19.75628 42.63384, 19.75005 42..."
1,CZ,0,CZ,Česko,Česko,0.0,0,0,"POLYGON ((14.33499 51.04007, 14.34494 51.03908..."
2,DE,0,DE,Deutschland,Deutschland,0.0,0,0,"MULTIPOLYGON (((10.45444 47.5558, 10.43954 47...."
3,DK,0,DK,Danmark,Danmark,0.0,0,0,"MULTIPOLYGON (((15.19309 55.32015, 15.19056 55..."
4,CY,0,CY,Kýpros,Κύπρος,0.0,0,0,"MULTIPOLYGON (((34.60609 35.70767, 34.6006 35...."
...,...,...,...,...,...,...,...,...,...
2005,NO0B1,3,NO,Jan Mayen,Jan Mayen,3.0,3,1,"POLYGON ((-9.08295 70.86469, -9.07843 70.86831..."
2006,EE009,3,EE,Kesk-Eesti,Kesk-Eesti,4.0,3,1,"MULTIPOLYGON (((25.98343 59.62115, 25.99837 59..."
2007,NO0,1,NO,Norge,Norge,0.0,0,0,"MULTIPOLYGON (((33.09131 80.24908, 33.09929 80..."
2008,NO0B,2,NO,Jan Mayen and Svalbard,Jan Mayen and Svalbard,NaN,0,0,"MULTIPOLYGON (((31.83002 80.07989, 31.85352 80..."


In [7]:
nuts3_regions = nuts[(nuts['LEVL_CODE'] == 3) & (nuts['CNTR_CODE'] == "UK")]
nuts3_regions = nuts3_regions[['NUTS_ID', 'CNTR_CODE', 'NAME_LATN']]
nuts3_regions.rename(columns={'NUTS_ID': 'nuts_cd', 'CNTR_CODE': 'country_cd', 'NAME_LATN': 'nuts_name'}, inplace=True)
nuts3_regions

,nuts_cd,country_cd,nuts_name
1417,UKM92,UK,Dumfries & Galloway
1574,UKK13,UK,Gloucestershire
1575,UKK14,UK,Swindon
1576,UKL22,UK,Cardiff and Vale of Glamorgan
1577,UKF14,UK,Nottingham
...,...,...,...
1999,UKJ43,UK,Kent Thames Gateway
2000,UKN08,UK,"Newry, Mourne and Down"
2001,UKM64,UK,Na h-Eileanan Siar (Western Isles)
2002,UKG37,UK,Sandwell


In [8]:
nuts3 = create_region_year_df('2010', '2023', nuts3_regions)
nuts3

,nuts_cd,country_cd,nuts_name,year,date
0,UKM92,UK,Dumfries & Galloway,2010,2010-12-31
1,UKM92,UK,Dumfries & Galloway,2011,2011-12-31
2,UKM92,UK,Dumfries & Galloway,2012,2012-12-31
3,UKM92,UK,Dumfries & Galloway,2013,2013-12-31
4,UKM92,UK,Dumfries & Galloway,2014,2014-12-31
...,...,...,...,...,...
2501,UKM77,UK,Perth & Kinross and Stirling,2019,2019-12-31
2502,UKM77,UK,Perth & Kinross and Stirling,2020,2020-12-31
2503,UKM77,UK,Perth & Kinross and Stirling,2021,2021-12-31
2504,UKM77,UK,Perth & Kinross and Stirling,2022,2022-12-31


In [9]:
nuts3 = nuts3.drop(columns=['country_cd', 'nuts_name', 'year'])
nuts3

,nuts_cd,date
0,UKM92,2010-12-31
1,UKM92,2011-12-31
2,UKM92,2012-12-31
3,UKM92,2013-12-31
4,UKM92,2014-12-31
...,...,...
2501,UKM77,2019-12-31
2502,UKM77,2020-12-31
2503,UKM77,2021-12-31
2504,UKM77,2022-12-31


In [10]:
GHI_nuts3 = disaggregate_climate_from_nuts2_to_nuts3(nuts3, GHI)
TP_nuts3 = disaggregate_climate_from_nuts2_to_nuts3(nuts3, TP)
MSL_nuts3 = disaggregate_climate_from_nuts2_to_nuts3(nuts3, MSL)
TA_nuts3 = disaggregate_climate_from_nuts2_to_nuts3(nuts3, TA)
WS_nuts3 = disaggregate_climate_from_nuts2_to_nuts3(nuts3, WS)



In [11]:
# Create a dictionary of DataFrames
dataframes = {'GHI_nuts3': GHI_nuts3, 'TP_nuts3': TP_nuts3, 'MSL_nuts3': MSL_nuts3, 'TA_nuts3': TA_nuts3, 'WS_nuts3': WS_nuts3}
# save as csv
save_data(dataframes, date="20241127")